# Import Libraries & Data

In [1]:
import os
import glob
import re
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_pickle('dolos_openface_merged.pkl')

In [3]:
dataset

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,file_name,label,video_id
0,1,0,0.00,0.88,1,-0.266795,0.265201,-0.926547,-0.384724,0.253047,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
1,2,0,0.04,0.88,1,-0.298252,0.246420,-0.922129,-0.410799,0.239001,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
2,3,0,0.08,0.88,1,-0.300153,0.241749,-0.922749,-0.413097,0.234598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
3,4,0,0.12,0.88,1,-0.300693,0.241151,-0.922730,-0.412985,0.233968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
4,5,0,0.16,0.98,1,0.276003,0.192164,-0.941751,0.125905,0.171836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AN_WILTY_EP15_lie4.csv,lie,AN_WILTY_EP15_lie4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190800,146,0,5.80,0.93,1,0.220692,-0.050918,-0.974014,0.050902,-0.056871,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth,YW_WILTY_EP70_truth16
190801,147,0,5.84,0.98,1,0.233077,-0.042794,-0.971516,0.053544,-0.046702,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth,YW_WILTY_EP70_truth16
190802,148,0,5.88,0.98,1,0.235445,-0.043778,-0.970901,0.054714,-0.047773,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth,YW_WILTY_EP70_truth16
190803,149,0,5.92,0.98,1,0.236376,-0.041068,-0.970793,0.054574,-0.044504,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,YW_WILTY_EP70_truth16.csv,truth,YW_WILTY_EP70_truth16


# Aggregate all data again

In [4]:
# at first we used only the AU column -> no good results so we added features for pose and gaze as well
#dataset.columns.values

In [5]:
dataset.sort_values(['video_id', 'face_id', 'timestamp'], inplace = True)

In [6]:
# keep only au, pose & gaze columns

au_cols = [c for c in dataset.columns if c.endswith("_r")]

pose_cols = [c for c in dataset.columns if c.startswith("pose_")]

gaze_cols = [c for c in dataset.columns if c.startswith("gaze_")]

final_columns = au_cols + pose_cols + pose_cols
#final_columns

## Compute aggregates (mean/max/std) per video

In [7]:
# compute aggregates (mean/max/std) per video pt.1

# compute  mean per video
video_mean = (dataset.groupby('video_id')[final_columns].mean())
video_mean

# compute AU mean per video
video_max = (dataset.groupby('video_id')[final_columns].max())
video_max

# compute AU mean per video
video_std = (dataset.groupby('video_id')[final_columns].std())
video_std

,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,AU14_r,...,pose_Tz,pose_Rx,pose_Ry,pose_Rz,pose_Tx,pose_Ty,pose_Tz,pose_Rx,pose_Ry,pose_Rz
video_id,,,,,,,,,,,,,,,,,,,,,
AN_WILTY_EP15_lie10,0.131243,0.085327,0.562402,0.033261,0.132124,0.181448,0.057571,0.446373,0.352630,0.328930,...,360.916228,0.084786,0.498067,0.071552,174.032103,69.130514,360.916228,0.084786,0.498067,0.071552
AN_WILTY_EP15_lie11,0.314456,0.219906,0.000000,0.130074,0.005639,0.000000,0.095572,0.297845,0.164786,0.574241,...,11.860857,0.060232,0.058292,0.065948,23.471155,9.319948,11.860857,0.060232,0.058292,0.065948
AN_WILTY_EP15_lie12,0.095647,0.070883,0.217000,0.079565,0.381883,0.461531,0.253215,0.538801,0.348212,0.470164,...,6.395445,0.051518,0.036717,0.035931,5.857887,4.081321,6.395445,0.051518,0.036717,0.035931
AN_WILTY_EP15_lie13,0.095652,0.076767,0.000000,0.048791,0.109656,0.129917,0.066048,0.419778,0.430428,0.564710,...,6.893409,0.031178,0.050602,0.050558,10.382623,4.785112,6.893409,0.031178,0.050602,0.050558
AN_WILTY_EP15_lie14,0.385574,0.348092,0.000000,0.113538,0.440128,0.313956,0.082497,0.566972,0.704785,0.606506,...,113.857880,0.070308,0.172690,0.041356,67.525139,20.701165,113.857880,0.070308,0.172690,0.041356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
YW_WILTY_EP70_truth5,0.147764,0.145458,0.101270,0.152342,0.292310,0.311208,0.064355,0.373886,0.475253,0.197343,...,4.847969,0.057337,0.063470,0.061649,10.083449,5.163165,4.847969,0.057337,0.063470,0.061649
YW_WILTY_EP70_truth6,0.527518,0.200851,0.521999,0.367288,0.429061,0.612170,0.167712,0.636275,0.572024,0.434525,...,89.687619,0.340680,0.408428,0.178965,52.138436,24.575249,89.687619,0.340680,0.408428,0.178965
YW_WILTY_EP70_truth7,0.436879,0.281196,0.361892,0.057731,0.114382,0.298365,0.099114,0.140731,0.228547,0.000000,...,58.006466,0.055712,0.126236,0.032992,44.884650,14.471699,58.006466,0.055712,0.126236,0.032992


In [8]:
# bring label column as well

data_temp = (dataset[['video_id', 'face_id', 'label']].drop_duplicates(subset = 'video_id'))

dataset_st_mean = video_mean.merge(data_temp, on = 'video_id')
dataset_st_mean

dataset_st_max = video_max.merge(data_temp, on = 'video_id')
dataset_st_max

dataset_st_std = video_std.merge(data_temp, on = 'video_id')
dataset_st_std

,video_id,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,AU12_r,...,pose_Ry,pose_Rz,pose_Tx,pose_Ty,pose_Tz,pose_Rx,pose_Ry,pose_Rz,face_id,label
0,AN_WILTY_EP15_lie10,0.131243,0.085327,0.562402,0.033261,0.132124,0.181448,0.057571,0.446373,0.352630,...,0.498067,0.071552,174.032103,69.130514,360.916228,0.084786,0.498067,0.071552,0,lie
1,AN_WILTY_EP15_lie11,0.314456,0.219906,0.000000,0.130074,0.005639,0.000000,0.095572,0.297845,0.164786,...,0.058292,0.065948,23.471155,9.319948,11.860857,0.060232,0.058292,0.065948,0,lie
2,AN_WILTY_EP15_lie12,0.095647,0.070883,0.217000,0.079565,0.381883,0.461531,0.253215,0.538801,0.348212,...,0.036717,0.035931,5.857887,4.081321,6.395445,0.051518,0.036717,0.035931,0,lie
3,AN_WILTY_EP15_lie13,0.095652,0.076767,0.000000,0.048791,0.109656,0.129917,0.066048,0.419778,0.430428,...,0.050602,0.050558,10.382623,4.785112,6.893409,0.031178,0.050602,0.050558,0,lie
4,AN_WILTY_EP15_lie14,0.385574,0.348092,0.000000,0.113538,0.440128,0.313956,0.082497,0.566972,0.704785,...,0.172690,0.041356,67.525139,20.701165,113.857880,0.070308,0.172690,0.041356,0,lie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1464,YW_WILTY_EP70_truth5,0.147764,0.145458,0.101270,0.152342,0.292310,0.311208,0.064355,0.373886,0.475253,...,0.063470,0.061649,10.083449,5.163165,4.847969,0.057337,0.063470,0.061649,0,truth
1465,YW_WILTY_EP70_truth6,0.527518,0.200851,0.521999,0.367288,0.429061,0.612170,0.167712,0.636275,0.572024,...,0.408428,0.178965,52.138436,24.575249,89.687619,0.340680,0.408428,0.178965,0,truth
1466,YW_WILTY_EP70_truth7,0.436879,0.281196,0.361892,0.057731,0.114382,0.298365,0.099114,0.140731,0.228547,...,0.126236,0.032992,44.884650,14.471699,58.006466,0.055712,0.126236,0.032992,0,truth
1467,YW_WILTY_EP70_truth8,0.301876,0.182715,0.172107,0.656657,0.452309,0.531807,0.094904,0.408353,0.513274,...,0.157272,0.136141,17.375231,6.708019,7.234302,0.097613,0.157272,0.136141,0,truth


## Compute temporal aggregates (mean/max/var) per video

In [9]:
# compute aggregates (mean/max/std) per video pt.2

grouped = dataset.groupby(['video_id', 'face_id', 'label'], sort = False)

# create temporal diff for AU columns
for c in au_cols:
    dataset[f"dau_{c}"] = grouped[c].diff().fillna(0)
    dataset[f"adau_{c}"] = grouped[f"dau_{c}"].transform(np.abs)

# create temporal diff for gaze/pose columns
for c in au_cols:
    dataset[f"dgp_{c}"] = grouped[c].diff().fillna(0)
    dataset[f"adgp_{c}"] = grouped[f"dgp_{c}"].transform(np.abs)

dataset

,frame,face_id,timestamp,confidence,success,gaze_0_x,gaze_0_y,gaze_0_z,gaze_1_x,gaze_1_y,...,dgp_AU20_r,adgp_AU20_r,dgp_AU23_r,adgp_AU23_r,dgp_AU25_r,adgp_AU25_r,dgp_AU26_r,adgp_AU26_r,dgp_AU45_r,adgp_AU45_r
175,1,0,0.00,0.88,1,-0.727288,0.130060,-0.673897,-0.791371,0.141323,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
176,2,0,0.04,0.88,1,-0.723201,0.131791,-0.677946,-0.789408,0.141825,...,0.00,0.00,0.04,0.04,0.00,0.00,0.27,0.27,-0.25,0.25
177,3,0,0.08,0.88,1,-0.718665,0.123627,-0.684278,-0.786161,0.133253,...,0.11,0.11,-0.05,0.05,0.10,0.10,0.14,0.14,-0.22,0.22
178,4,0,0.12,0.88,1,-0.715956,0.105691,-0.690099,-0.785690,0.116391,...,0.12,0.12,-0.02,0.02,0.00,0.00,-0.14,0.14,0.06,0.06
179,5,0,0.16,0.88,1,-0.708524,0.083249,-0.700759,-0.782517,0.095179,...,0.17,0.17,-0.09,0.09,0.00,0.00,-0.18,0.18,-0.09,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189950,71,0,2.80,0.98,1,0.242643,0.147295,-0.958868,0.048420,0.149139,...,0.18,0.18,0.00,0.00,-0.21,0.21,-0.20,0.20,0.12,0.12
189951,72,0,2.84,0.98,1,0.251441,0.168185,-0.953148,0.055887,0.171904,...,0.12,0.12,0.00,0.00,-0.09,0.09,-0.13,0.13,0.12,0.12
189952,73,0,2.88,0.98,1,0.254587,0.183241,-0.949531,0.058724,0.187430,...,0.03,0.03,0.00,0.00,0.09,0.09,0.07,0.07,0.03,0.03
189953,74,0,2.92,0.98,1,0.252872,0.193974,-0.947856,0.056238,0.197261,...,-0.04,0.04,0.00,0.00,0.16,0.16,0.18,0.18,-0.09,0.09


In [10]:
#dataset.columns.values

In [11]:
# keep only diff_au, adiff_au, diff_gp & adiff_gp columns

diff_au_cols = [c for c in dataset.columns if c.startswith('dau_')]

adiff_au_cols = [c for c in dataset.columns if c.startswith('adau_')]

diff_gp_cols = [c for c in dataset.columns if c.startswith('dgp_')]

adiff_gp_cols = [c for c in dataset.columns if c.startswith('adgp_')]

#final_columns = au_cols + pose_cols + pose_cols
#final_columns

In [12]:
# final step: group by & aggregate

# AU diffs
dau_mean = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].mean().add_prefix("dau_mean_")
dau_min = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].min().add_prefix("dau_min_")
dau_max = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].max().add_prefix("dau_max_")
dau_var = dataset.groupby(['video_id', 'face_id', 'label'])[diff_au_cols].var().add_prefix("dau_var_")

# AU absolute diffs
adau_mean = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].mean().add_prefix("adau_mean_")
adau_min = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].min().add_prefix("adau_min_")
adau_max = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].max().add_prefix("adau_max_")
adau_var = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_au_cols].var().add_prefix("adau_var_")

# gaze & pose diffs
dgp_mean = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].mean().add_prefix("dgp_mean_")
dgp_min = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].min().add_prefix("dgp_min_")
dgp_max = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].max().add_prefix("dgp_max_")
dgp_var = dataset.groupby(['video_id', 'face_id', 'label'])[diff_gp_cols].var().add_prefix("dgp_var_")

# gaze & pose absolute diffs
adgp_mean = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].mean().add_prefix("adgp_mean_")
adgp_min = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].min().add_prefix("adgp_min_")
adgp_max = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].max().add_prefix("adgp_max_")
adgp_var = dataset.groupby(['video_id', 'face_id', 'label'])[adiff_gp_cols].var().add_prefix("adgp_var_")

In [13]:
# union all aggregates in one table at last

agg_blocks = [
    dau_mean, dau_min, dau_max, dau_var, # AU diffs
    adau_mean, adau_min, adau_max, adau_var,  # AU absolute diffs
    dgp_mean, dgp_min, dgp_max, dgp_var,   # gaze & pose diffs
    adgp_mean, adgp_min, adgp_max, adgp_var # gaze & pose absolute diffs
]

for agg in agg_blocks:
    if agg.empty:
        continue
    data_temp = data_temp.merge(agg.reset_index(), on = ['video_id', 'face_id', 'label'], how = 'left')

# Output Files

#### ---> From now on we have one row per video with AU, Pose, Gaze mean/max/std & label

In [14]:
data_temp.columns

Index(['video_id', 'face_id', 'label', 'dau_mean_dau_AU01_r',
       'dau_mean_dau_AU02_r', 'dau_mean_dau_AU04_r', 'dau_mean_dau_AU05_r',
       'dau_mean_dau_AU06_r', 'dau_mean_dau_AU07_r', 'dau_mean_dau_AU09_r',
       ...
       'adgp_var_adgp_AU10_r', 'adgp_var_adgp_AU12_r', 'adgp_var_adgp_AU14_r',
       'adgp_var_adgp_AU15_r', 'adgp_var_adgp_AU17_r', 'adgp_var_adgp_AU20_r',
       'adgp_var_adgp_AU23_r', 'adgp_var_adgp_AU25_r', 'adgp_var_adgp_AU26_r',
       'adgp_var_adgp_AU45_r'],
      dtype='object', length=275)

In [15]:
data_temp.to_pickle('dolos_aggr_temporal_v2.pkl')

In [16]:
dataset_st_mean.to_pickle('dolos_aggr_mean_v2.pkl')
dataset_st_max.to_pickle('dolos_aggr_max_v2.pkl')
dataset_st_std.to_pickle('dolos_aggr_std_v2.pkl')